In [10]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:.3f}'.format
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings('ignore')

In [11]:
# cargando el dataset con todas las probabilidades de compra de cada cliente
# obtenidas con el modelo de machine learning en el notebook Recomendacion_part_1.ipynb
df_probab_compra = pd.read_parquet('df_proba_compra.parquet')
df_probab_compra.head()

,prob_compra
pk_cid,
15891,0.465
16063,0.697
16203,0.837
16502,0.811
17457,0.709


In [12]:
# cargando el dataset con las variables sociodemograficas de cada cliente desde S3
df_socio = pd.read_parquet("https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/sociodemographic_df.parquet")
df_socio.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.000,H,35,N,87218.100
1,1,1050611,2018-01-28,ES,13.000,V,23,N,35548.740
2,2,1050612,2018-01-28,ES,13.000,V,23,N,122179.110
3,3,1050613,2018-01-28,ES,50.000,H,22,N,119775.540
4,4,1050614,2018-01-28,ES,50.000,V,23,N,NaN


In [13]:
# cargando el dataset con las variables de productos de cada cliente
df_prod = pd.read_parquet("https://easy-money-project-bucket.s3.eu-west-3.amazonaws.com/products_df.parquet")
df_prod.head()

,Unnamed: 0,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1


In [14]:
# se hace un merge de los dos datasets anteriores para tener un dataset completo con todas las variables
df_full = pd.merge(df_prod, df_socio, on=["pk_cid","pk_partition"], how="left")
df_full.head() 

,Unnamed: 0_x,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,Unnamed: 0_y,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0,ES,29.000,H,35,N,87218.100
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,1,ES,13.000,V,23,N,35548.740
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,2,ES,13.000,V,23,N,122179.110
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,3,ES,50.000,H,22,N,119775.540
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,4,ES,50.000,V,23,N,NaN


Eliminamos todos los clientes que aparecen en deceased como S

In [15]:
# se han eliminado los clientes fallecidos, 86 en total
df_full = df_full[df_full["deceased"] != "S"]
df_full

,Unnamed: 0_x,pk_cid,pk_partition,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,Unnamed: 0_y,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0,ES,29.000,H,35,N,87218.100
1,1,1050611,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,1,ES,13.000,V,23,N,35548.740
2,2,1050612,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,2,ES,13.000,V,23,N,122179.110
3,3,1050613,2018-01-28,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,3,ES,50.000,H,22,N,119775.540
4,4,1050614,2018-01-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,4,ES,50.000,V,23,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,13647304,1166765,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647304,ES,50.000,V,22,N,43912.170
5962920,13647305,1166764,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647305,ES,26.000,V,23,N,23334.990
5962921,13647306,1166763,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647306,ES,50.000,H,47,N,NaN
5962922,13647307,1166789,2019-05-28,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,13647307,ES,50.000,H,22,N,199592.820


Ahora nos quedamos con las columnas que nos interesan para el analisis

In [16]:
columnas_relev = ['pk_cid','short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 
                  'long_term_deposit',"em_account_pp",	"credit_card"	,"payroll"	,"pension_plan",
                  "payroll_account"	,"emc_account",	"debit_card",	"em_account_p"	,"em_acount"]

df_full = df_full[columnas_relev]
df_full

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,1050611,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
2,1050612,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,1050613,1,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
4,1050614,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962920,1166764,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962921,1166763,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
5962922,1166789,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1


Agrupamos por cliente y nos quedamos con el valor maximo de cada variable, garantizando que si en algun periodo el cliente usa un producto pues se sobre entiende que tiene contratado dicho producto.

In [17]:
df_full = df_full.groupby("pk_cid").agg("max").reset_index()
df_full

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,15891,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
1,16063,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
2,16203,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1
3,16502,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1
4,17457,0,0,0,0,0,1,0,0,0.000,0.000,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456313,1553685,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456314,1553686,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456315,1553687,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0
456316,1553688,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0


# Enfoque Global

A continuacion se crea un ranking de productos para recomendar, basándonos en los que el cliente no tiene y ordenándolos según la probabilidad de compra. Esta estrategia permite mejorar la eficiencia de la campaña de marketing y aumentar la satisfacción y fidelización de los clientes.


In [18]:
# 1. Identificar las columnas de productos (todas excepto 'pk_cid')
product_cols = df_full.columns.drop('pk_cid')

# 2. Sumar los valores de cada columna de productos
product_totals = df_full[product_cols].sum()

# 3. Crear un DataFrame con los totales
product_ranking = product_totals.reset_index()
product_ranking.columns = ['Producto', 'Total']

# 4. Ordenar los productos por total en orden descendente
product_ranking = product_ranking.sort_values(by='Total', ascending=False)
product_ranking

,Producto,Total
14,em_acount,332686.000
12,debit_card,63521.000
10,payroll_account,31578.000
9,pension_plan,28197.000
11,emc_account,27624.000
8,payroll,26892.000
5,long_term_deposit,9101.000
7,credit_card,8527.000
0,short_term_deposit,5323.000
4,securities,2086.000


In [19]:
product_ranking["ranking"] = product_ranking["Total"].rank(ascending=False)
product_ranking

,Producto,Total,ranking
14,em_acount,332686.000,1.000
12,debit_card,63521.000,2.000
10,payroll_account,31578.000,3.000
9,pension_plan,28197.000,4.000
11,emc_account,27624.000,5.000
8,payroll,26892.000,6.000
5,long_term_deposit,9101.000,7.000
7,credit_card,8527.000,8.000
0,short_term_deposit,5323.000,9.000
4,securities,2086.000,10.000


Se crea un DataFrame que contiene para cada cliente (pk_cid) una lista de recomendaciones de productos basado en el ranking. Esto se realiza con la funcion get_top_n_recommendations, que identifica el producto de mayor ranking que el cliente no tiene y se sugiere como recomendación. A continuación se describe especificamente que hace la función:

1. Identifica los productos que el cliente no tiene.
2. Se crea un nuevo ranking de esos productos (que el cliente no tiene), basado en el Ranking principal
3. Selecciona los productos primeros en el ranking y se retornar en una lista de 3 parametros(dichos prametros se pueden cambiar y poner directamente n=1 y te devuelve un solo producto)

In [20]:

def get_top_n_recommendations(row, n=3):
    # Obtener los productos que el cliente tiene
    client_products = row[product_cols]
     # Identificar los productos que el cliente NO tiene
    not_owned_products = client_products[client_products == 0].index.tolist()
    # Filtrar el ranking para incluir solo los productos que el cliente no tiene
    ranking = product_ranking.set_index('Producto')['ranking']
    not_owned_ranking = ranking.loc[not_owned_products]
     # Verificar si hay productos que recomendar
    if not not_owned_ranking.empty:
        # Obtener los top N productos con el ranking más alto
        recommended_products = not_owned_ranking.nsmallest(n).index.tolist()
        return recommended_products
    else:
        return None  # El cliente tiene todos los productos

# Aplicar la función para obtener las top N recomendaciones
df_full['recomendaciones'] = df_full.apply(get_top_n_recommendations, axis=1, n=3)

# Crear el DataFrame final con las recomendaciones
df_recommendations = df_full[['pk_cid', 'recomendaciones']]

# Mostrar el DataFrame de recomendaciones
print(df_recommendations)

         pk_cid                               recomendaciones
0         15891   [debit_card, payroll_account, pension_plan]
1         16063      [em_acount, debit_card, payroll_account]
2         16203   [debit_card, payroll_account, pension_plan]
3         16502   [debit_card, payroll_account, pension_plan]
4         17457  [payroll_account, pension_plan, emc_account]
...         ...                                           ...
456313  1553685      [em_acount, debit_card, payroll_account]
456314  1553686      [em_acount, debit_card, payroll_account]
456315  1553687      [em_acount, debit_card, payroll_account]
456316  1553688      [em_acount, debit_card, payroll_account]
456317  1553689      [em_acount, debit_card, payroll_account]

[456318 rows x 2 columns]


Para quedarnos con la primera recomendación

In [21]:
df_recommendations["recomend"] = df_recommendations["recomendaciones"].apply(lambda x: x[0] if x else None)
df_recommendations.head()

,pk_cid,recomendaciones,recomend
0,15891,"[debit_card, payroll_account, pension_plan]",debit_card
1,16063,"[em_acount, debit_card, payroll_account]",em_acount
2,16203,"[debit_card, payroll_account, pension_plan]",debit_card
3,16502,"[debit_card, payroll_account, pension_plan]",debit_card
4,17457,"[payroll_account, pension_plan, emc_account]",payroll_account


Ahora se agrega el precio de cada producto

In [22]:
# Definir las listas de productos
cuenta_bancaria_products = ['em_account_pp', 'payroll', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']
inversion_products = ['short_term_deposit', 'funds', 'securities', 'long_term_deposit', 'pension_plan']
financiacion_products = ['loans', 'mortgage', 'credit_card']

In [23]:
products_precios = {
    'em_account_pp': 10, 'payroll': 10, 'payroll_account': 10, 'emc_account': 10, 'debit_card': 10, 'em_account_p': 10, 'em_acount': 10,
    'short_term_deposit': 40, 'funds': 40, 'securities': 40, 'long_term_deposit': 40, 'pension_plan': 40,
    'loans': 60, 'mortgage': 60, 'credit_card': 60
}

df_recommendations["precio"] = df_recommendations["recomend"].apply(lambda x: products_precios[x] if x else None)
df_recommendations

,pk_cid,recomendaciones,recomend,precio
0,15891,"[debit_card, payroll_account, pension_plan]",debit_card,10
1,16063,"[em_acount, debit_card, payroll_account]",em_acount,10
2,16203,"[debit_card, payroll_account, pension_plan]",debit_card,10
3,16502,"[debit_card, payroll_account, pension_plan]",debit_card,10
4,17457,"[payroll_account, pension_plan, emc_account]",payroll_account,10
...,...,...,...,...
456313,1553685,"[em_acount, debit_card, payroll_account]",em_acount,10
456314,1553686,"[em_acount, debit_card, payroll_account]",em_acount,10
456315,1553687,"[em_acount, debit_card, payroll_account]",em_acount,10
456316,1553688,"[em_acount, debit_card, payroll_account]",em_acount,10


In [24]:
df_recommendations.value_counts("recomend")

recomend
debit_card            278215
em_acount             123632
payroll_account        41702
emc_account             9353
long_term_deposit       1714
pension_plan            1475
credit_card               90
payroll                   76
short_term_deposit        55
securities                 6
Name: count, dtype: int64

In [25]:
df_recommendations["precio"].value_counts()

precio
10    452978
40      3250
60        90
Name: count, dtype: int64

Análisis de los Resultados:

- Concentración de Recomendaciones: La gran mayoría de las recomendaciones se concentran en los primeros tres productos del ranking global, que a su vez representan al grupo de productos Cuentas.

- Poca Diversidad en Recomendaciones: Los demás productos reciben muy pocas recomendaciones en comparación.

- Posible Falta de Relevancia Personalizada: Este enfoque global no considera las diferencias individuales entre los clientes.

Limitaciones del Enfoque Global:

- Falta de Personalización: Todos los clientes reciben recomendaciones basadas en el mismo ranking, sin considerar sus características individuales. Es probable que las necesidades y preferencias varíen significativamente entre diferentes grupos de clientes.
  
- Recomendaciones Poco Relevantes: Algunos clientes pueden no estar interesados en los productos más recomendados. Productos que podrían ser más relevantes para ciertos clientes están siendo ignorados debido al ranking global.

	
- Oportunidades Perdidas: Al no segmentar, se podría estar perdiendo oportunidades para ofrecer productos que tienen mayor potencial de aceptación en ciertos grupos.



Teniendo en cuenta todo lo anterior se toma la decision de hacer un analisis de ranking de productos por grupos, a partir de la segmentación obtenida en la Tarea 2:

- Se adaptan las recomendaciones según las características y necesidades específicas de cada grupo lo que aumenta la probabilidad de aceptación.

- Mayor y mejor diversidad en recomendaciones ya que productos que son menos recomendados en el enfoque global podrían ser más relevantes para ciertos grupos y, por tanto, recomendados con mayor frecuencia.
  
Por lo tanto, implementar un análisis de recomendaciones por grupos de clientes mejorará significativamente la relevancia de las recomendaciones y se aprovechará mejor el potencial del catálogo de productos.

---

# Enfoque por grupos de clientes

Cargando el dataset de la segmentación de la Tarea 2

In [26]:
df_clustering = pd.read_parquet("s3://easy-money-project-bucket/df_clustering.parquet")
df_clustering

,num_products_contracts,entry_date,active_customer,mes_partition,age,cuentas,ahorro_inversion,financiacion,entry_channel_group_Canal Secundario,salary_category_Ingreso Bajo,salary_category_Ingreso Medio,cluster_6
pk_cid,,,,,,,,,,,,
15891,0.000,0.486,0.500,3.000,2.429,-0.773,0.000,0.000,1.000,0.000,0.000,3
16063,-1.000,0.641,0.714,0.000,2.643,-1.545,0.000,0.000,1.000,0.000,0.000,5
16203,0.000,0.688,0.833,0.000,3.214,-0.258,0.000,0.000,1.000,0.000,0.000,4
16502,1.000,0.573,0.889,0.000,2.357,0.687,0.000,0.000,1.000,0.000,0.000,4
17457,2.000,0.058,1.000,0.000,2.071,1.455,0.000,0.074,1.000,0.000,1.000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1553685,-1.000,0.904,0.000,0.000,1.929,-1.545,0.000,0.000,1.000,0.000,0.000,5
1553686,-1.000,0.904,0.000,0.000,0.357,-1.545,0.000,0.000,1.000,0.000,1.000,2
1553687,-1.000,0.904,0.000,0.000,-0.286,-1.545,0.000,0.000,1.000,0.000,0.000,2


In [27]:
# Uniendo el DataFrame de recomendaciones con el DataFrame de clustering
df_full = pd.merge(df_full, df_clustering["cluster_6"], on="pk_cid", how="left")
df_full.drop("recomendaciones", axis=1, inplace=True)

In [28]:
df_full

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,cluster_6
0,15891,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,3
1,16063,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,5
2,16203,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,4
3,16502,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,1,4
4,17457,0,0,0,0,0,1,0,0,0.000,0.000,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456313,1553685,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,5
456314,1553686,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,2
456315,1553687,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,2
456316,1553688,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,0,5


In [29]:
df_full["cluster_6"].value_counts()

cluster_6
0    245139
2     73062
4     63037
1     35232
5     32586
3      7262
Name: count, dtype: int64

Diviediendo el dataset en varios dataset por los grupos existentes

In [30]:
df_cluster_0 = df_full[df_full["cluster_6"] == 0]
df_cluster_1 = df_full[df_full["cluster_6"] == 1]
df_cluster_2 = df_full[df_full["cluster_6"] == 2]
df_cluster_3 = df_full[df_full["cluster_6"] == 3]
df_cluster_4 = df_full[df_full["cluster_6"] == 4]
df_cluster_5 = df_full[df_full["cluster_6"] == 5]


In [31]:
df_cluster_0

,pk_cid,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,cluster_6
272,84263,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
366,106231,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
576,148606,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
595,151847,0,0,0,0,0,0,0,0,0.000,0.000,0,1,0,0,0,0
688,169963,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456023,1553303,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
456088,1553393,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
456120,1553429,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0
456142,1553456,0,0,0,0,0,0,0,0,0.000,0.000,0,0,0,0,1,0


In [32]:
# 1. Identificar las columnas de productos (todas excepto 'pk_cid')
product_cols = ['short_term_deposit', 'loans', 'mortgage', 'funds', 'securities',
                'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll',
       'pension_plan', 'payroll_account', 'emc_account', 'debit_card',
       'em_account_p', 'em_acount']

dicc_cluster = {"cluster_0": df_cluster_0, "cluster_1": df_cluster_1,
                 "cluster_2": df_cluster_2, "cluster_3": df_cluster_3, 
                 "cluster_4": df_cluster_4, "cluster_5": df_cluster_5}

df_rankind_cluster = pd.DataFrame()

for i,j in dicc_cluster.items():
       # 2. Sumar los valores de cada columna de productos
       product_totals = j[product_cols].sum()
       
       # 3. Crear un DataFrame con los totales
       product_ranking = product_totals.reset_index()
       product_ranking.columns = [f'Producto_{i}', f'Total_{i}']
       
       # 4. Ordenar los productos por total en orden descendente
       product_ranking.sort_values(by=f'Total_{i}', ascending=False, inplace=True, ignore_index=True)
       
       df_rankind_cluster = pd.concat([df_rankind_cluster, product_ranking], axis=1)
       #df_rankind_cluster.reset_index(drop=True, inplace=True)

df_rankind_cluster

,Producto_cluster_0,Total_cluster_0,Producto_cluster_1,Total_cluster_1,Producto_cluster_2,Total_cluster_2,Producto_cluster_3,Total_cluster_3,Producto_cluster_4,Total_cluster_4,Producto_cluster_5,Total_cluster_5
0,em_acount,243107.000,payroll_account,26857.000,em_acount,2378.000,em_acount,2439.000,em_acount,57640.000,em_acount,1889.000
1,debit_card,15903.000,pension_plan,26569.000,debit_card,97.000,debit_card,503.000,debit_card,21320.000,credit_card,374.000
2,emc_account,2875.000,payroll,25925.000,emc_account,62.000,emc_account,279.000,emc_account,11778.000,emc_account,324.000
3,payroll_account,1902.000,debit_card,25563.000,credit_card,60.000,short_term_deposit,244.000,long_term_deposit,4999.000,short_term_deposit,265.000
4,short_term_deposit,408.000,em_acount,25233.000,short_term_deposit,27.000,payroll_account,157.000,short_term_deposit,3623.000,debit_card,135.000
5,long_term_deposit,301.000,emc_account,12306.000,payroll_account,20.000,pension_plan,128.000,payroll_account,2610.000,payroll_account,32.000
6,pension_plan,269.000,credit_card,6846.000,pension_plan,16.000,payroll,115.000,pension_plan,1210.000,securities,24.000
7,credit_card,157.000,long_term_deposit,3688.000,securities,6.000,long_term_deposit,92.000,credit_card,1056.000,long_term_deposit,19.000
8,payroll,153.000,securities,1316.000,loans,2.000,credit_card,34.000,payroll,696.000,funds,17.000
9,securities,122.000,funds,1070.000,long_term_deposit,2.000,securities,6.000,securities,612.000,pension_plan,5.000


Se puede observar que el analisis del ranking de los productos según los grupos varía es más especifico con respecto al enfoque general.

Los productos más populares varían entre clusters. Por ejemplo:

- payroll_account es el más popular en Cluster 1 pero no lo es en los demás.
- credit_card es el segundo más popular en Cluster 2, pero no tiene la misma posición en otros clusters.
- En algunos clusters, ciertos productos tienen totales muy bajos o incluso cero, indicando poca o ninguna adopción, como se observa en el cluster_3

Los datos muestran claramente que los clientes en diferentes clusters tienen preferencias y comportamientos distintos. Esto fundamenta la necesidad de hacer el análisis por grupos.

## Recomendación para cada cluster

Agrupando los ranking por cluster

In [55]:
ranking_cluster_0 = df_rankind_cluster[["Producto_cluster_0", "Total_cluster_0"]]
ranking_cluster_1 = df_rankind_cluster[["Producto_cluster_1", "Total_cluster_1"]]
ranking_cluster_2 = df_rankind_cluster[["Producto_cluster_2", "Total_cluster_2"]]
ranking_cluster_3 = df_rankind_cluster[["Producto_cluster_3", "Total_cluster_3"]]
ranking_cluster_4 = df_rankind_cluster[["Producto_cluster_4", "Total_cluster_4"]]
ranking_cluster_5 = df_rankind_cluster[["Producto_cluster_5", "Total_cluster_5"]]

ranking_cluster_0["ranking"] = ranking_cluster_0["Total_cluster_0"].rank(ascending=False)
ranking_cluster_1["ranking"] = ranking_cluster_1["Total_cluster_1"].rank(ascending=False)
ranking_cluster_2["ranking"] = ranking_cluster_2["Total_cluster_2"].rank(ascending=False)
ranking_cluster_3["ranking"] = ranking_cluster_3["Total_cluster_3"].rank(ascending=False)
ranking_cluster_4["ranking"] = ranking_cluster_4["Total_cluster_4"].rank(ascending=False)
ranking_cluster_5["ranking"] = ranking_cluster_5["Total_cluster_5"].rank(ascending=False)


Utilizando la misma funcion que para el enfoque general, se obtiene la recomendación de cada producto para cada cliente por grupos.

In [56]:
dicc_cluster_ranking = {0:ranking_cluster_0, 1:ranking_cluster_1,
                2:ranking_cluster_2, 3:ranking_cluster_3,
                4:ranking_cluster_4, 5:ranking_cluster_5}


for i,j in dicc_cluster_ranking.items():
    
    def get_top_n_recommendations(row, n=3):
        # Obtener los productos que el cliente tiene
        client_products = row[product_cols]
        # Identificar los productos que el cliente NO tiene
        not_owned_products = client_products[client_products == 0].index.tolist()
        # Filtrar el ranking para incluir solo los productos que el cliente no tiene
        ranking = j.set_index(f'Producto_cluster_{i}')['ranking']
        not_owned_ranking = ranking.loc[not_owned_products]
        # Verificar si hay productos que recomendar
        if not not_owned_ranking.empty:
            # Obtener los top N productos con el ranking más alto
            recommended_products = not_owned_ranking.nsmallest(n).index.tolist()
            return recommended_products
        else:
            return None
        
    df_cluster = dicc_cluster[f"cluster_{i}"]
    # Aplicar la función para obtener las top N recomendaciones
    df_cluster[f'recomendaciones_{i}'] = df_cluster.apply(get_top_n_recommendations, axis=1, n=1)
    # Crear el DataFrame final con las recomendaciones
    if i == 0:
        df_cluster_recomend_0 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_0[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_0.head())
    elif i == 1:
        df_cluster_recomend_1 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print  (df_cluster_recomend_1[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_1.head())
    elif i == 2:
        df_cluster_recomend_2 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_2[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_2.head())
    elif i == 3:
        df_cluster_recomend_3 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_3[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_3.head())
    elif i == 4:
        df_cluster_recomend_4 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_4[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_4.head())
    elif i == 5:
        df_cluster_recomend_5 = df_cluster[['pk_cid', f'recomendaciones_{i}']]
        print(df_cluster_recomend_5[f"recomendaciones_{i}"].value_counts())
        print(df_cluster_recomend_5.head())




recomendaciones_0
[debit_card]            227308
[emc_account]            15746
[em_acount]               2032
[payroll_account]           47
[short_term_deposit]         6
Name: count, dtype: int64
     pk_cid recomendaciones_0
272   84263      [debit_card]
366  106231      [debit_card]
576  148606      [debit_card]
595  151847       [em_acount]
688  169963      [debit_card]
recomendaciones_1
[emc_account]           8930
[payroll_account]       8375
[em_acount]             7755
[debit_card]            6712
[credit_card]           1366
[pension_plan]           972
[payroll]                625
[long_term_deposit]      436
[securities]              47
[funds]                   13
[short_term_deposit]       1
Name: count, dtype: int64
    pk_cid  recomendaciones_1
4    17457  [payroll_account]
7    17970        [em_acount]
19   20333  [payroll_account]
38   26018  [payroll_account]
39   26163  [payroll_account]
recomendaciones_2
[em_acount]      70684
[debit_card]      2324
[emc_account] 

Como se puede observar, por cada grupo de cliente se obtiene la recomendación de producto para cada cliente.

## Incluyendo las probabilidades, precio y beneficio en cada cluster

Mergeando los dataset de recomendacion por cada grupo con su probabilidad de compra

Se agrega para cada cluster el precio de compra del producto recomendado, el beneficio (prprob_compra * precio) y por ultimo el cluster a que pertenece.

In [60]:
df_cluster_recomend_0 = df_cluster_recomend_0.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_0["precio"] = df_cluster_recomend_0["recomendaciones_0"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_0["beneficio"] = df_cluster_recomend_0["prob_compra"] * df_cluster_recomend_0["precio"]
df_cluster_recomend_0["cluster"] = 0
df_cluster_recomend_0

,pk_cid,recomendaciones_0,prob_compra,precio,beneficio,cluster
0,84263,[debit_card],0.763,10,7.631,0
1,106231,[debit_card],0.758,10,7.583,0
2,148606,[debit_card],0.682,10,6.823,0
3,151847,[em_acount],0.730,10,7.302,0
4,169963,[debit_card],0.593,10,5.934,0
...,...,...,...,...,...,...
245134,1553303,[debit_card],0.242,10,2.419,0
245135,1553393,[debit_card],0.080,10,0.798,0
245136,1553429,[debit_card],0.221,10,2.212,0
245137,1553456,[debit_card],0.252,10,2.523,0


In [61]:
df_cluster_recomend_1 = df_cluster_recomend_1.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_1["precio"] = df_cluster_recomend_1["recomendaciones_1"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_1["beneficio"] = df_cluster_recomend_1["prob_compra"] * df_cluster_recomend_1["precio"]
df_cluster_recomend_1["cluster"] = 1
df_cluster_recomend_1

,pk_cid,recomendaciones_1,prob_compra,precio,beneficio,cluster
0,17457,[payroll_account],0.709,10,7.093,1
1,17970,[em_acount],0.703,10,7.028,1
2,20333,[payroll_account],0.885,10,8.853,1
3,26018,[payroll_account],0.953,10,9.528,1
4,26163,[payroll_account],0.706,10,7.058,1
...,...,...,...,...,...,...
35227,1551114,[payroll_account],0.197,10,1.970,1
35228,1551480,[payroll_account],0.318,10,3.176,1
35229,1551712,[payroll_account],0.177,10,1.765,1
35230,1552055,[payroll_account],0.189,10,1.885,1


In [63]:
df_cluster_recomend_2 = df_cluster_recomend_2.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_2["precio"] = df_cluster_recomend_2["recomendaciones_2"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_2["beneficio"] = df_cluster_recomend_2["prob_compra"] * df_cluster_recomend_2["precio"]
df_cluster_recomend_2["cluster"] = 2
df_cluster_recomend_2

,pk_cid,recomendaciones_2,prob_compra,precio,beneficio,cluster
0,491473,[em_acount],0.069,10,0.694,2
1,528596,[em_acount],0.237,10,2.368,2
2,541741,[em_acount],0.175,10,1.745,2
3,583277,[em_acount],0.551,10,5.508,2
4,614292,[em_acount],0.763,10,7.631,2
...,...,...,...,...,...,...
73057,1553673,[em_acount],0.221,10,2.212,2
73058,1553677,[em_acount],0.221,10,2.212,2
73059,1553679,[em_acount],0.142,10,1.420,2
73060,1553686,[em_acount],0.160,10,1.604,2


In [64]:
df_cluster_recomend_3 = df_cluster_recomend_3.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_3["precio"] = df_cluster_recomend_3["recomendaciones_3"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_3["beneficio"] = df_cluster_recomend_3["prob_compra"] * df_cluster_recomend_3["precio"]
df_cluster_recomend_3["cluster"] = 3
df_cluster_recomend_3

,pk_cid,recomendaciones_3,prob_compra,precio,beneficio,cluster
0,15891,[debit_card],0.465,10,4.655,3
1,28470,[em_acount],0.294,10,2.939,3
2,35063,[debit_card],0.929,10,9.290,3
3,53660,[debit_card],0.497,10,4.968,3
4,61348,[em_acount],0.294,10,2.939,3
...,...,...,...,...,...,...
7257,1520003,[em_acount],0.177,10,1.765,3
7258,1520017,[em_acount],0.221,10,2.213,3
7259,1520058,[em_acount],0.187,10,1.873,3
7260,1520126,[em_acount],0.178,10,1.782,3


In [67]:
df_cluster_recomend_4 = df_cluster_recomend_4.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_4["precio"] = df_cluster_recomend_4["recomendaciones_4"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_4["beneficio"] = df_cluster_recomend_4["prob_compra"] * df_cluster_recomend_4["precio"]
df_cluster_recomend_4["cluster"] = 4
df_cluster_recomend_4

,pk_cid,recomendaciones_4,prob_compra,precio,beneficio,cluster
0,16203,[debit_card],0.837,10,8.365,4
1,16502,[debit_card],0.811,10,8.112,4
2,17799,[debit_card],0.709,10,7.093,4
3,17980,[em_acount],0.925,10,9.252,4
4,18146,[em_acount],0.727,10,7.274,4
...,...,...,...,...,...,...
63032,1553541,[debit_card],0.273,10,2.725,4
63033,1553559,[debit_card],0.139,10,1.386,4
63034,1553565,[debit_card],0.166,10,1.656,4
63035,1553620,[em_acount],0.111,10,1.110,4


In [66]:
df_cluster_recomend_5 = df_cluster_recomend_5.merge(df_probab_compra, on="pk_cid", how="left")
df_cluster_recomend_5["precio"] = df_cluster_recomend_5["recomendaciones_5"].apply(lambda x: products_precios[x[0]] if x else None)
df_cluster_recomend_5["beneficio"] = df_cluster_recomend_5["prob_compra"] * df_cluster_recomend_5["precio"]
df_cluster_recomend_5["cluster"] = 5
df_cluster_recomend_5


,pk_cid,recomendaciones_5,prob_compra,precio,beneficio,cluster
0,16063,[em_acount],0.697,10,6.968,5
1,17590,[em_acount],0.758,10,7.583,5
2,19812,[em_acount],0.691,10,6.913,5
3,20303,[em_acount],0.734,10,7.338,5
4,22491,[em_acount],0.739,10,7.390,5
...,...,...,...,...,...,...
32581,1553681,[em_acount],0.186,10,1.859,5
32582,1553683,[em_acount],0.252,10,2.523,5
32583,1553685,[em_acount],0.102,10,1.025,5
32584,1553688,[em_acount],0.318,10,3.176,5


Se renombra la columna recomendaciones para evitar problemas y se unen en un mismo dataframe los df anteriores.

In [74]:
df_cluster_recomend_0.rename(columns={"recomendaciones_0":"recomendacion"}, inplace=True)
df_cluster_recomend_1.rename(columns={"recomendaciones_1":"recomendacion"}, inplace=True)
df_cluster_recomend_2.rename(columns={"recomendaciones_2":"recomendacion"}, inplace=True)
df_cluster_recomend_3.rename(columns={"recomendaciones_3":"recomendacion"}, inplace=True)
df_cluster_recomend_4.rename(columns={"recomendaciones_4":"recomendacion"}, inplace=True)
df_cluster_recomend_5.rename(columns={"recomendaciones_5":"recomendacion"}, inplace=True)

# quiero unir los 6 dataframes anteriores en uno solo, pero uno debajo del otro
df_recommendations_final = pd.concat([
    df_cluster_recomend_0,
    df_cluster_recomend_1,
    df_cluster_recomend_2,
    df_cluster_recomend_3,
    df_cluster_recomend_4,
    df_cluster_recomend_5
], ignore_index=True)
df_recommendations_final

,pk_cid,recomendacion,prob_compra,precio,beneficio,cluster
0,84263,[debit_card],0.763,10,7.631,0
1,106231,[debit_card],0.758,10,7.583,0
2,148606,[debit_card],0.682,10,6.823,0
3,151847,[em_acount],0.730,10,7.302,0
4,169963,[debit_card],0.593,10,5.934,0
...,...,...,...,...,...,...
456313,1553681,[em_acount],0.186,10,1.859,5
456314,1553683,[em_acount],0.252,10,2.523,5
456315,1553685,[em_acount],0.102,10,1.025,5
456316,1553688,[em_acount],0.318,10,3.176,5


## Análisis de los 10mil clientes

En este analisis se van a tener en cuenta 3 escenarios

Partimos primeramente a partir del comprotamiento de los cluster obtenidos en la tarea 2

In [119]:
# Datos para la tabla
data_summary = {
    "Cluster": [
        "Clientes Exploradores Iniciales", 
        "Clientes de Alto Valor", 
        "Clientes Pasivos o en Pausa",
        "Clientes Emergentes", 
        "Clientes Senior en Expansión", 
        "Clientes Senior Tradicionalistas"
    ],
    "Número de Productos Contratados": [
        "Bajo", 
        "Alto", 
        "Muy Bajo", 
        "Bajo", 
        "Moderado", 
        "Muy Bajo"
    ],
    "Actividad del Cliente": [
        "Moderada", 
        "Muy Alta", 
        "Muy Baja", 
        "Media Baja", 
        "Alta", 
        "Muy Baja"
    ],
    "Ingreso": [
        "Ingreso Bajo/Medio", 
        "Ingreso Medio Alto", 
        "Ingreso Medio Bajo", 
        "Ingreso Medio", 
        "Mixto/Medio Alto", 
        "Medio Alto"
    ],
    "Canal de Entrada": [
        "Bajo uso de secundarios", 
        "Media alta en secundarios", 
        "Muy bajo uso de secundarios", 
        "Alta en secundarios", 
        "Alta en secundarios", 
        "Alta en secundarios"
    ],
    "Edad": [
        "Mixta", 
        "Mediana Edad", 
        "Jóvenes", 
        "Mediana Edad", 
        "Mayores", 
        "Mayores"
    ],
    "Uso de Cuentas, Ahorro e Inversión": [
        "Bajo", 
        "Alto", 
        "Muy Bajo", 
        "Moderado/Bajo", 
        "Moderado-Alto/Bajo", 
        "Muy Bajo"
    ],
    "Descripción Breve": [
        "Clientes con actividad moderada y potencial para explorar más productos.",
        "Clientes leales, usando múltiples productos financieros.",
        "Clientes jóvenes con baja actividad y compromiso.",
        "Clientes de mediana edad en fase de transición y crecimiento.",
        "Clientes mayores con compromiso y potencial para más productos.",
        "Clientes mayores con enfoque en métodos tradicionales y básicos."
    ],
    "Estrategias Clave": [
        "Cuentas sin comisiones, recompensas, asesoría gratuita.",
        "Programas de lealtad personalizados, servicios financieros avanzados.",
        "Incentivos para productos básicos.",
        "Ventas cruzadas, educación financiera.",
        "Productos de ahorro/inversión para mayores.",
        "Productos financieros básicos adaptados a mayores."
    ]
}

# Crear el dataframe
df_summary = pd.DataFrame(data_summary)

# Mostrar la tabla
df_summary

,Cluster,Número de Productos Contratados,Actividad del Cliente,Ingreso,Canal de Entrada,Edad,"Uso de Cuentas, Ahorro e Inversión",Descripción Breve,Estrategias Clave
0,Clientes Exploradores Iniciales,Bajo,Moderada,Ingreso Bajo/Medio,Bajo uso de secundarios,Mixta,Bajo,Clientes con actividad moderada y potencial pa...,"Cuentas sin comisiones, recompensas, asesoría ..."
1,Clientes de Alto Valor,Alto,Muy Alta,Ingreso Medio Alto,Media alta en secundarios,Mediana Edad,Alto,"Clientes leales, usando múltiples productos fi...","Programas de lealtad personalizados, servicios..."
2,Clientes Pasivos o en Pausa,Muy Bajo,Muy Baja,Ingreso Medio Bajo,Muy bajo uso de secundarios,Jóvenes,Muy Bajo,Clientes jóvenes con baja actividad y compromiso.,Incentivos para productos básicos.
3,Clientes Emergentes,Bajo,Media Baja,Ingreso Medio,Alta en secundarios,Mediana Edad,Moderado/Bajo,Clientes de mediana edad en fase de transición...,"Ventas cruzadas, educación financiera."
4,Clientes Senior en Expansión,Moderado,Alta,Mixto/Medio Alto,Alta en secundarios,Mayores,Moderado-Alto/Bajo,Clientes mayores con compromiso y potencial pa...,Productos de ahorro/inversión para mayores.
5,Clientes Senior Tradicionalistas,Muy Bajo,Muy Baja,Medio Alto,Alta en secundarios,Mayores,Muy Bajo,Clientes mayores con enfoque en métodos tradic...,Productos financieros básicos adaptados a mayo...



cuenta_bancaria_products = ['em_account_pp', 'payroll', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']

inversion_products = ['short_term_deposit', 'funds', 'securities', 'long_term_deposit', 'pension_plan']

financiacion_products = ['loans', 'mortgage', 'credit_card']

### Clientes que ofrecen mayor beneficio

En este escenario se ordenan los clientes que brindan mayor beneficio para la empresa y se seleccionan los primeros 10mil

In [75]:
df_recommendations_final = df_recommendations_final.sort_values(by='beneficio', ascending=False)
df_seleccionados = df_recommendations_final.head(10000)

In [76]:
df_seleccionados["cluster"].value_counts()

cluster
1    4047
4    2064
0    1904
5    1890
2      53
3      42
Name: count, dtype: int64

In [122]:
df_seleccionados[df_seleccionados["cluster"] == 4]["recomendacion"].value_counts()

recomendacion
debit_card            1167
long_term_deposit      528
em_acount              242
emc_account             80
short_term_deposit      45
payroll_account          2
Name: count, dtype: int64

In [123]:
df_seleccionados["recomendacion"].value_counts()

recomendacion
credit_card           3252
debit_card            3029
pension_plan           972
long_term_deposit      964
payroll_account        798
em_acount              578
emc_account            227
short_term_deposit      76
securities              47
payroll                 44
funds                   13
Name: count, dtype: int64

In [105]:
df_seleccionados.sum()

pk_cid                                                 12028851419
recomendacion    credit_cardcredit_cardcredit_cardcredit_cardcr...
prob_compra                                               8368.397
precio                                                      324760
beneficio                                               244498.501
cluster                                                      21985
dtype: object

In [126]:
df_seleccionados.sort_values(by='prob_compra', ascending=True)

,pk_cid,recomendacion,prob_compra,precio,beneficio,cluster
447759,1512633,credit_card,0.161,60,9.672,5
448253,1517902,credit_card,0.172,60,10.345,5
448131,1516836,credit_card,0.188,60,11.283,5
278240,1505779,credit_card,0.191,60,11.471,1
444872,1400320,credit_card,0.198,60,11.904,5
...,...,...,...,...,...,...
255006,1121611,em_acount,0.993,10,9.931,1
373790,1136095,debit_card,0.993,10,9.931,4
256945,1137152,payroll_account,0.993,10,9.931,1
372393,1126036,debit_card,0.993,10,9.931,4


Como resultado de este escenario, se puede observar que se presenta una adecuada distribución de productos segun el comportamiento de los cluster y como se puede ver por ejemplo en el cluster 4 estan presentes muchos productos de inversion lo cual esta en correlacion con la estrategia clave de dicho cluster.

Además, el beneficio alcanzado por esta campaña sería como mínimo de 244498.501 euros y como máximo de 324760 euros.

Por otra parte, se considera que este escenario no es el más adecuado ya que tiende a maximizar el beneficio y no tiene mucho en cuenta las probalidades de compra, es decir, presenta clientes con probabilidades de compra muy bajas pero como tiene recomendado un producto con alto precio pues la ganancia seria mayor con respecto a otros.

### Clientes con mayor probabilidades de compra

En este escenario se realiza un analisis similar al anterior pero a partir de las probabilidades de compra, ordenando los clientes que presentan mayor probalidad de compra y nos quedamos con los 10mil primeros.

In [107]:
los_10000 = df_recommendations_final.sort_values(by='prob_compra', ascending=False).head(10000)
los_10000.sum()

pk_cid                                                 11907095302
recomendacion    [debit_card, payroll_account, em_acount, payro...
prob_compra                                               9611.225
precio                                                      107970
beneficio                                               103824.882
cluster                                                      11238
dtype: object

In [108]:
los_10000["cluster"].value_counts()

cluster
0    5635
4    2186
1    1962
2     155
3      44
5      18
Name: count, dtype: int64

In [109]:
los_10000["recomendacion"].value_counts()

recomendacion
[debit_card]            7104
[payroll_account]       1041
[em_acount]              927
[emc_account]            657
[credit_card]             76
[pension_plan]            64
[payroll]                 56
[long_term_deposit]       45
[securities]              13
[short_term_deposit]      11
[funds]                    6
Name: count, dtype: int64

En este escenario se puede observar un poco de desbalance a la hora de recomendar los productos principalmente debit card y lo mismo sucede a la hora de distribuir productos por cluster, ademas el beneficio que se puede lograr alcanzar es bajo siendo como minimo 103824.882 y como maximo 107970. A pesar de que seria un escenerio basatnate realista por las probalidades de compra altas no esta garantizando un benificio adecuado.


### Analisis mixto de clientes. 

En este escenario se analiza el comportamiento de la probablidad de compra por cada establecer los umbrales de probabilidad de compra adecuados para cada grupo. A continuación, analizaremos los datos de cada cluster y definiremos los umbrales basándonos en la distribución de las probabilidades de compra (prob_compra).

In [112]:
df_recommendations_final[df_recommendations_final["cluster"] ==0].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,245139.000,245139.000,245139.000,245139.000,245139.000
mean,1260377.115,0.806,10.001,8.057,0.000
std,154778.917,0.149,0.148,1.490,0.000
min,84263.000,0.008,10.000,0.078,0.000
25%,1126015.000,0.756,10.000,7.564,0.000
50%,1267493.000,0.868,10.000,8.683,0.000
75%,1411640.500,0.897,10.000,8.973,0.000
max,1553571.000,0.990,40.000,36.314,0.000


- Alta Probabilidad de Compra Promedio: La media de 0.806 indica que, en promedio, los clientes de este cluster tienen una probabilidad del 80.6% de comprar el producto recomendado.

- Umbral Alto: Dado que la mayoría de las probabilidades son altas, podemos establecer un umbral más alto para enfocarnos en los clientes con mayor probabilidad.

- Propuesta de Umbral: 0.80. Esto incluiría aproximadamente al 50% de los clientes (ya que la mediana es 0.868). Nos enfocamos en clientes con una probabilidad de compra del 80% o más.

In [113]:
df_recommendations_final[df_recommendations_final["cluster"] ==1].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,35232.000,35232.000,35232.000,35232.000,35232.000
mean,1226843.010,0.736,13.189,9.744,1.000
std,186967.318,0.144,11.148,8.701,0.000
min,17457.000,0.029,10.000,0.287,1.000
25%,1113838.500,0.643,10.000,6.474,1.000
50%,1238776.000,0.724,10.000,7.379,1.000
75%,1370696.250,0.868,10.000,8.948,1.000
max,1552364.000,0.993,60.000,59.527,1.000


- Probabilidad de Compra Promedio Alta: Media de 0.736.

- Umbral Moderado-Alto: Dado que son clientes de alto valor, podemos permitir un umbral ligeramente más bajo para incluir más clientes potenciales.

- Propuesta de Umbral: 0.70. Incluiría a más del 50% de los clientes. Aceptable dado su alto valor y potencial.

In [115]:
df_recommendations_final[df_recommendations_final["cluster"] ==2].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,73062.000,73062.000,73062.000,73062.000,73062.000
mean,1313329.191,0.570,10.001,5.697,2.000
std,153635.842,0.241,0.185,2.414,0.000
min,491473.000,0.007,10.000,0.070,2.000
25%,1199338.500,0.374,10.000,3.743,2.000
50%,1336681.500,0.595,10.000,5.951,2.000
75%,1462571.500,0.792,10.000,7.921,2.000
max,1553687.000,0.965,60.000,41.748,2.000


- Probabilidad de Compra Promedio Moderada: Media de 0.570.

- Umbral Más Alto: Para enfocarnos en los clientes más propensos a comprar.

- Propuesta de Umbral: 0.60. Incluiría a los clientes por encima de la mediana. Enfoca recursos en clientes con mayor probabilidad dentro de un cluster pasivo.

In [116]:
df_recommendations_final[df_recommendations_final["cluster"] ==3].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,7262.000,7262.000,7262.000,7262.000,7262.000
mean,1372692.561,0.434,10.099,4.407,3.000
std,167461.096,0.284,1.722,3.139,0.000
min,15891.000,0.008,10.000,0.078,3.000
25%,1337504.500,0.176,10.000,1.761,3.000
50%,1429323.000,0.419,10.000,4.194,3.000
75%,1473981.500,0.685,10.000,6.849,3.000
max,1520374.000,0.988,40.000,39.004,3.000


- Probabilidad de Compra Promedio Baja: Media de 0.434.

- Umbral Alto: Enfocarse en clientes con probabilidad significativamente alta.

- Propuesta de Umbral: 0.60. Se enfoca en el 25% superior de clientes con mayor probabilidad.

In [117]:
df_recommendations_final[df_recommendations_final["cluster"] ==4].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,63037.000,63037.000,63037.000,63037.000,63037.000
mean,1294983.049,0.702,10.276,7.219,4.000
std,229184.494,0.139,2.864,2.554,0.000
min,16203.000,0.019,10.000,0.185,4.000
25%,1182366.000,0.634,10.000,6.355,4.000
50%,1354175.000,0.694,10.000,6.954,4.000
75%,1453827.000,0.778,10.000,7.820,4.000
max,1553684.000,0.993,40.000,39.478,4.000


- Probabilidad de Compra Promedio Alta: Media de 0.702.

- Umbral Moderado-Alto: Para incluir a clientes con buena probabilidad.

- Propuesta de Umbral: 0.70. Se enfoca en clientes por encima de la media.

In [118]:
df_recommendations_final[df_recommendations_final["cluster"] ==5].describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,32586.000,32586.000,32586.000,32586.000,32586.000
mean,1290182.880,0.598,12.895,7.930,5.000
std,243522.086,0.171,11.679,8.400,0.000
min,16063.000,0.053,10.000,0.530,5.000
25%,1131669.000,0.569,10.000,5.687,5.000
50%,1336774.000,0.640,10.000,6.430,5.000
75%,1517200.500,0.694,10.000,7.070,5.000
max,1553689.000,0.969,60.000,57.989,5.000


- Probabilidad de Compra Promedio Moderada: Media de 0.598.
- Umbral Moderado: Para incluir a clientes con probabilidad por encima de la media.
- Propuesta de Umbral: 0.60

In [127]:
# tabla resumen de umbrales por cluster
umbrales = { "Clientes Exploradores Iniciales": f"> 0.80",
             "Clientes de Alto Valor": f"> 0.70",
             "Clientes Pasivos o en Pausa": f"> 0.60",
             "Clientes Emergentes": f"> 0.60",
             "Clientes Senior en Expansión": f"> 0.70",
             "Clientes Senior Tradicionalistas": f"> 0.60"
           }

umbrales_df = pd.DataFrame(umbrales.items(), columns=["Cluster", "Umbral de Probabilidad de Compra"])
umbrales_df


,Cluster,Umbral de Probabilidad de Compra
0,Clientes Exploradores Iniciales,> 0.80
1,Clientes de Alto Valor,> 0.70
2,Clientes Pasivos o en Pausa,> 0.60
3,Clientes Emergentes,> 0.60
4,Clientes Senior en Expansión,> 0.70
5,Clientes Senior Tradicionalistas,> 0.60


Aplicaremos un filtrado por umbrales de cluster al df_recommendations_final

In [128]:
umbrales_por_cluster = {
    0: 0.80,
    1: 0.70,
    2: 0.60,
    3: 0.60,
    4: 0.70,
    5: 0.60
}

clusters_filtrados = []

# Filtrar los clientes por cluster y umbral de probabilidad de compra en df_recommendations_final
for i, df_cluster in df_recommendations_final.groupby("cluster"):
    umbral = umbrales_por_cluster.get(i)
    df_cluster_filtrado = df_cluster[df_cluster['prob_compra'] >= umbral]
    clusters_filtrados.append(df_cluster_filtrado)

"for i, df_cluster in enumerate(clusters):\n    umbral = umbrales_por_cluster.get(i)\n    df_cluster_filtrado = df_cluster[df_cluster['prob_compra'] >= umbral]\n    clusters_filtrados.append(df_cluster_filtrado)"

In [131]:
# llevando clusters filtrados a un solo dataframe
df_clusters_filtrados = pd.concat(clusters_filtrados, ignore_index=True)
df_clusters_filtrados

,pk_cid,recomendacion,prob_compra,precio,beneficio,cluster
0,1377217,[short_term_deposit],0.908,40,36.314,0
1,1086587,[short_term_deposit],0.904,40,36.152,0
2,1078963,[debit_card],0.990,10,9.903,0
3,1089851,[debit_card],0.989,10,9.887,0
4,1066229,[debit_card],0.989,10,9.887,0
...,...,...,...,...,...,...
281536,1225227,[em_acount],0.600,10,6.002,5
281537,1541705,[em_acount],0.600,10,6.001,5
281538,1539131,[em_acount],0.600,10,6.001,5
281539,1544943,[em_acount],0.600,10,6.001,5


In [132]:
# comprobando comportamiento de los clusters filtrados segun la probabilidad de compra
df_clusters_filtrados.describe()

,pk_cid,prob_compra,precio,beneficio,cluster
count,281541.000,281541.000,281541.000,281541.000,281541.000
mean,1256105.454,0.844,10.529,8.853,1.150
std,171871.921,0.083,4.870,3.821,1.700
min,16063.000,0.600,10.000,6.001,0.000
25%,1124765.000,0.812,10.000,8.148,0.000
50%,1259722.000,0.870,10.000,8.704,0.000
75%,1402959.000,0.900,10.000,9.013,2.000
max,1548217.000,0.993,60.000,59.527,5.000


A continuacion, se ordena por beneficio esperado y seleccionamos los 10mil clientes

In [133]:
df_todos_clusters = df_clusters_filtrados.sort_values(by='beneficio', ascending=False)
df_seleccionados = df_todos_clusters.head(10000)
distribucion_clusters_final = df_seleccionados['cluster'].value_counts()
print("Distribución final de clientes por cluster:")
print(distribucion_clusters_final)

Distribución final de clientes por cluster:
cluster
0    3170
1    3124
4    2106
5    1479
2      77
3      44
Name: count, dtype: int64


- Cluster 0 (“Clientes Exploradores Iniciales”) aporta el mayor número de clientes, con 3,170 clientes (31.7% del total).
- Cluster 1 (“Clientes de Alto Valor”) aporta 3,124 clientes (31.24% del total).
- Cluster 4 (“Clientes Senior en Expansión”) aporta 2,106 clientes (21.06% del total).
- Cluster 5 (“Clientes Senior Tradicionalistas”) aporta 1,479 clientes (14.79% del total).
- Clusters 2 y 3 aportan una cantidad mínima de clientes, lo cual es coherente con sus características y los umbrales aplicados.

In [134]:
df_seleccionados["recomendacion"].value_counts()

recomendacion
[debit_card]            4497
[credit_card]           2290
[payroll_account]        935
[em_acount]              710
[long_term_deposit]      530
[pension_plan]           526
[emc_account]            377
[payroll]                 47
[short_term_deposit]      44
[securities]              33
[funds]                   11
Name: count, dtype: int64

- debit_card es el producto más recomendado, con 4,497 clientes (44.97% del total).
- credit_card es el segundo más recomendado, con 2,290 clientes (22.9% del total).
- Los productos de mayor precio, como credit_card, long_term_deposit, pension_plan, están bien representados.
- Los productos de menor precio, como debit_card, em_acount, emc_account, también tienen una alta presencia, lo que refleja un buen equilibrio.

In [136]:
df_seleccionados.sum()

pk_cid                                                 11994717633
recomendacion    [credit_card, credit_card, credit_card, credit...
prob_compra                                               9042.295
precio                                                      248820
beneficio                                               206088.223
cluster                                                      19229
dtype: object

- El beneficio total esperado de la campaña como mínimo es de 206088.223, basado en las probabilidades de compra y los precios de los productos y como máximo seria de 248820.
  


In [137]:
df_seleccionados[df_seleccionados["cluster"] == 0]["recomendacion"].value_counts()    

recomendacion
[debit_card]            2961
[emc_account]            163
[em_acount]               43
[short_term_deposit]       2
[payroll_account]          1
Name: count, dtype: int64

- La mayoría de los clientes de este cluster recibieron recomendaciones de [debit_card], lo cual es coherente con sus características: clientes con baja contratación de productos y actividad moderada.
  
- Estrategia Adecuada: Ofrecer productos básicos y de fácil acceso para incentivar su actividad y explorar más productos.

In [138]:
df_seleccionados[df_seleccionados["cluster"] == 1]["recomendacion"].value_counts() 

recomendacion
[payroll_account]      932
[credit_card]          815
[pension_plan]         526
[em_acount]            297
[long_term_deposit]    265
[debit_card]           107
[emc_account]           91
[payroll]               47
[securities]            33
[funds]                 11
Name: count, dtype: int64

- Este cluster tiene una distribución diversa de productos recomendados, incluyendo productos de mayor precio como [credit_card], [pension_plan], [long_term_deposit], y [securities].
  
- Estrategia Adecuada: Ofrecer productos avanzados y personalizados que reflejen su alto valor y capacidad adquisitiva.


In [139]:
df_seleccionados[df_seleccionados["cluster"] == 2]["recomendacion"].value_counts() 

recomendacion
[em_acount]      70
[debit_card]      6
[credit_card]     1
Name: count, dtype: int64

In [140]:
df_seleccionados[df_seleccionados["cluster"] == 3]["recomendacion"].value_counts() 

recomendacion
[short_term_deposit]    17
[debit_card]            14
[em_acount]              7
[emc_account]            6
Name: count, dtype: int64

Para los cluster 2 y 3:

- Baja Representación: Coherente con sus características y los umbrales aplicados.
  
- Estrategia Adecuada: Enfocarse en clientes con mayor probabilidad de compra dentro de estos clusters, ofreciendo productos básicos para incentivar su actividad.

In [141]:
df_seleccionados[df_seleccionados["cluster"] == 4]["recomendacion"].value_counts() 

recomendacion
[debit_card]            1409
[em_acount]              288
[long_term_deposit]      265
[emc_account]            117
[short_term_deposit]      25
[payroll_account]          2
Name: count, dtype: int64

- La mayoría de las recomendaciones son [debit_card] y [em_acount], pero también hay presencia significativa de [long_term_deposit].
  
- Estrategia Adecuada: Enfocarse en productos de ahorro e inversión adaptados a clientes mayores, resaltando seguridad y estabilidad.

In [142]:
df_seleccionados[df_seleccionados["cluster"] == 5]["recomendacion"].value_counts() 

recomendacion
[credit_card]    1474
[em_acount]         5
Name: count, dtype: int64

- La gran mayoría de los clientes recibieron la recomendación de [credit_card], un producto de mayor precio.
  
- Estrategia Adecuada: Dado su perfil conservador pero con capacidad adquisitiva, ofrecer productos que puedan brindarles beneficios claros y seguridad financiera.

Conclusión

- Este escenario presenta mejor comportamiento que los anteriores.
  
- La seleccion de los clientes ha sido basada en datos, aplicando umbrales de probabilidad adecuados basados en el análisis estadístico de cada cluster.

- La distribución de clientes entre los clusters es coherente y equilibrada.
- Las recomendaciones de productos se alinean con las características y necesidades de cada cluster.



In [144]:
# Guardar el DataFrame de recomendaciones seleccionadas para los 10000 clientes
df_seleccionados.to_parquet("df_seleccionados.parquet")